# Install and use CPlantBox in 5 minutes
In this tutorial, you only need to click the small triangle button on the left top corner of the code blocks.
Feel free to modify the code if needed, remember to save the notebook by download or save in google drive, if you note down all the modifications, you don't have to download the entire CPlantBox or the modified parameter files.
In the following block is the library we needed for running CPlantBox

In [65]:
#<---Click this triangle to load python libraries
# loading other python library or packages. Only need to run once at the start.
import datetime
import matplotlib.pylab as plt
import numpy as np
import timeit
import os
import plotly
import os
import math
import plotly.graph_objects as go
import xml.etree.ElementTree as ET
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d
from plotly.subplots import make_subplots
!pip3 install vtk
from vtk.util import numpy_support as VN

  Using cached https://files.pythonhosted.org/packages/22/f5/30e11e1ad21701e1cd185b046979107930419a74a5602c6b899dc8523fe4/vtk-8.1.2-cp36-cp36m-manylinux1_x86_64.whl


## 1. Install CPlantBox
* The source code of CPlantBox is always published on GitHub, here we can download them (git clone) from github
* The next thing after download is to compile it, compile is similar to installation, you can move on.

In [ ]:
#<---Click this triangle inside the round circle to download the CPlantBox
! git clone https://github.com/Plant-Root-Soil-Interactions-Modelling/CPlantBox # downloading the source code
os.chdir("/content/CPlantBox") # change directory
!cmake . # Run Cmake, creating compiling files
!make # Complie the code to make it runable

# Congrats! Now you are ready to use the CPlantBox

# optional in case an update is just pushed to the repository
#! git pull # update the file comparing to github repository

## 2. Load CPlantBox

In [2]:
os.chdir("/content/CPlantBox/python") # Change to the python directory (working directory)
# Loading specific python scripts for CPlantBox and CRootBox
import py_plantbox as pb #CPlantBox Python Binding
from rb_tools import * # CRootBox nodes Conversion tools
from CPlantBox_PiafMunch import *
plotly.__version__
# If not all of them are installed, run the following blocks to install them, and rerun this block
# Otherwise skip the next block


'4.1.1'

## 3. Creating a small plant and visiualize it

1. Three things needed for a CPlantBox function/method are the name of the input parameter, time for the simulation and output name of the file.
    * Name must be the same as the file in the /modelparameter folder, you can learn how to create it in the next section
    * Time is the time that plant grown, normally larger number means 

In [114]:
# first we create a Heliantus plant
name = "Heliantus_Pagès_2013" # parameter name
# here are some optional parameters to be tested
# name = "PMA2018" # Simulate a small plant with 3 leaves and two lateral root, you can comment the heliantus line and uncomment this line to see what happend.
time = 70 # how many days the plant need to grow, make it smaller, for example 15 to see if the plant becomes smaller

plant1 = CPlantBox(name, time, name) # make a plant object in python
# Visualization
def visual_plant_sub(plant1,name='plant'):
    nodes_cor = python_nodes(plant1) # use the object name created previously to get its coordinates
    fig= go.Scatter3d(
        x=nodes_cor.T[3],
        y=nodes_cor.T[4],
        z=nodes_cor.T[5],
        mode='markers',
        marker=dict(
            size=3,
            color=nodes_cor.T[1],                # nodes_cor.T[1] is organ type, nodes_cor.T[2] is the connection number of a node 
            colorscale=[[0, "wheat"], #color of the root, change it to "yellow" to see the difference
                    [0.5, "darkgreen"],
                    [1.0, "lightgreen"],],  opacity=0.8
        ), name =name
    )

    return fig
   

def visual_plant(plant1):
    subfig = visual_plant_sub(plant1)
    fig = make_subplots(
    rows=1, cols=1,
    specs=[[{'type': 'surface'}]])
    fig.add_trace(subfig) 
    fig.update_layout(scene_aspectmode='data',)
    return fig

fig = visual_plant(plant1)
fig.show()



## 4. Read and write parameter files.
1. CPlantBox use XML file to parameterize the plant, the file could be a little bit scarry at first glance. However, here I will give you examples to try each parameter. In this way you will know all the parameters quickly.
Firstly, we print out all the parameters:

In [103]:
name= "PMA2018" #name of the small plant
all_parameter = ET.parse("../modelparameter/{}.xml".format(name)) # read the parameter file from xml file
plant_parameter = all_parameter.getroot() # get the first level of parameters
parameter_options={}
parameter_order = 0
for organ in plant_parameter.iter('organ'): 
    list=[]
    print(organ.attrib ) # print of parameters
    
    for parameter in organ.iter('parameter'): 
        
        print(parameter.attrib)
        
        list.append(parameter.attrib['name']) 
        parameter_order = parameter_order+1
    print("\n" )
    parameter_options[organ.attrib['type']+organ.attrib['subType']] = list

{'type': 'seed', 'subType': '0'}
{'name': 'seedPos.x', 'value': '0'}
{'name': 'seedPos.y', 'value': '0'}
{'name': 'seedPos.z', 'value': '-3'}
{'name': 'plantingdepth', 'value': '3'}
{'name': 'firstB', 'value': '0', 'dev': '0'}
{'name': 'delayB', 'value': '0', 'dev': '0'}
{'name': 'maxB', 'value': '0', 'dev': '0'}
{'name': 'nC', 'value': '2'}
{'name': 'nz', 'value': '0'}
{'name': 'firstSB', 'value': '14', 'dev': '0'}
{'name': 'delaySB', 'value': '0', 'dev': '0'}
{'name': 'delayRC', 'value': '7', 'dev': '0'}
{'name': 'simulationTime', 'value': '56', 'dev': '0'}


{'type': 'root', 'name': 'taproot', 'subType': '1'}
{'name': 'lb', 'value': '1', 'dev': '0'}
{'name': 'la', 'value': '2', 'dev': '0'}
{'name': 'ln', 'value': '3', 'dev': '0'}
{'name': 'lmax', 'value': '4.0999999999999996', 'dev': '0'}
{'name': 'nob', 'value': '1.3666666666666665', 'dev': '0'}
{'name': 'r', 'value': '4.5', 'dev': '0.45000000000000001'}
{'name': 'a', 'value': '0.20000000000000001', 'dev': '0.02'}
{'name': 'RotBeta

In [104]:
# here we show what is the plant simulated based on those parameters
name= "PMA2018"
smallplant = CPlantBox(name, time, name) # make a plant object in python
fig = visual_plant(smallplant)    
fig.show()

## 5. Change the length of the Stem
### 5.1 lmax: the maximal length
Length is controled by six parameters, among them, the maximal length is controled by lmax. if we only change the lmax, the growth will only depends on the time. Lets see what will happend if we only change the lmax of the main stem:

In [105]:
def change_parameter(input_name, output_name, organ_name, subtype , parameter_name, value_type, value):   
    all_parameter = ET.parse("../modelparameter/{}.xml".format(input_name)) # read the parameter file from xml file
    plant_parameter = all_parameter.getroot() # get the first level of parameters
    original_lmax = plant_parameter.find("./organ[@type='{}'][@subType='{}']/parameter[@name='{}']".format(organ_name, subtype, parameter_name)).get('{}'.format(value_type)) # get function to read the value
    # The '10' in the following line is the value that need to be changed change the value
    plant_parameter.find("./organ[@type='{}'][@subType='{}']/parameter[@name='{}']".format(organ_name, subtype, parameter_name)).set('{}'.format(value_type),'{}'.format(value)) # set function to read the value
    current_lmax = plant_parameter.find("./organ[@type='{}'][@subType='{}']/parameter[@name='{}']".format(organ_name, subtype, parameter_name)).get('{}'.format(value_type)) # get function to read the value
    print('original {} of {} organ with subtype {} is {}, changed to {}'.format(parameter_name, organ_name, subtype ,original_lmax, current_lmax))
    all_parameter.write('../modelparameter/{}.xml'.format(output_name))
change_parameter('PMA2018', 'testXML', 'stem', '1', 'lmax', 'value','10')
name = 'testXML'
smallplant = CPlantBox(name, time, name) # make a plant object in python
fig = visual_plant(smallplant)    
fig.show()
# Try to change the "10" to 5
change_parameter('PMA2018', 'testXML', 'stem', '1', 'lmax', 'value','5')
smallplant = CPlantBox(name, time, name) # make a plant object in python
fig = visual_plant(smallplant)    
fig.show()

original lmax of stem organ with subtype 1 is 2.1, changed to 10


original lmax of stem organ with subtype 1 is 2.1, changed to 5


### 5.2 lb, ln and la: the parameters controls branching
In the previous section, you may notice that, with the increase of the lmax, the length of the main stem increased. But with the increase length of the stem, we also get more leaves. Is it possible to reduce the leaf numbers? There are two ways, first, we could change the lb, lb is the length at bottom without any branching.

In [106]:
# change lb to 3 based on the testXML created in last section
name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'lb', 'value','3')
smallplant = CPlantBox(name, time, name) # make a plant object in python
fig = visual_plant(smallplant)    
fig.show()
# now there is 3 branches again

original lb of stem organ with subtype 1 is 0.7, changed to 3


### 5.2.2 Let's change the ln to see what will happen?
ln is the internode distance

In [55]:
name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'ln', 'value','0.3')
smallplant = CPlantBox(name, time, name) # make a plant object in python
fig = visual_plant(smallplant)    
fig.show()

original lmax is 0.5, changed to 0.3


### 5.2.2Let's change the la to see what will happen?
la is the stem top without branches

In [57]:
name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'la', 'value','2')
smallplant = CPlantBox(name, time, name) # make a plant object in python
fig = visual_plant(smallplant)    
fig.show()

original lmax is 1, changed to 2


In [64]:
# in addition, the functiontype decides what branching internode distance will be
name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'ln', 'functiontype','1')
smallplant = CPlantBox(name, time, name) # make a plant object in python
fig = visual_plant(smallplant)    
fig.show()

original ln of stem organ with subtype 1 is 2, changed to 1


In [126]:

fig = make_subplots(
    rows=3, cols=2,
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}],
           [{'type': 'scatter3d'}, {'type': 'scatter3d'}],
           [{'type': 'scatter3d'}, {'type': 'scatter3d'}]])

name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'lmax', 'value','20')
change_parameter('testXML', 'testXML', 'leaf', '2', 'RotBeta', 'value','1')
change_parameter('testXML', 'testXML', 'stem', '1', 'ln', 'functiontype','0')
smallplant1 = CPlantBox(name, time, name) # make a plant object in python
fig1 = visual_plant_sub(smallplant1,name="functiontype 0")

name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'ln', 'functiontype','1')
smallplant2 = CPlantBox(name, time, name) # make a plant object in python
fig2 = visual_plant_sub(smallplant2,name="functiontype 1")

name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'ln', 'functiontype','2')
smallplant3 = CPlantBox(name, time, name) # make a plant object in python
fig3 = visual_plant_sub(smallplant3,name="functiontype 2")

name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'ln', 'functiontype','3')
smallplant4 = CPlantBox(name, time, name) # make a plant object in python
fig4 = visual_plant_sub(smallplant4,name="functiontype 3")

name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'ln', 'functiontype','2')
smallplant5 = CPlantBox(name, time, name) # make a plant object in python
fig5 = visual_plant_sub(smallplant3,name="functiontype 4")

name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'ln', 'functiontype','3')
smallplant6 = CPlantBox(name, time, name) # make a plant object in python
fig6 = visual_plant_sub(smallplant4,name="functiontype 5")

fig.add_trace(fig1, 1, 1 )
fig.add_trace(fig2, 1, 2)
fig.add_trace(fig3, 2, 1)
fig.add_trace(fig4, 2, 2)
fig.add_trace(fig5, 3, 1)
fig.add_trace(fig6, 3, 2)

fig.update_layout(
    title_text='different RotBeta (function type)',
    height=800,
    width=800
)

fig.show()

original lmax of stem organ with subtype 1 is 20, changed to 20
original RotBeta of leaf organ with subtype 2 is 1, changed to 1
original ln of stem organ with subtype 1 is 3, changed to 0
original ln of stem organ with subtype 1 is 0, changed to 1
original ln of stem organ with subtype 1 is 1, changed to 2
original ln of stem organ with subtype 1 is 2, changed to 3
original ln of stem organ with subtype 1 is 3, changed to 2
original ln of stem organ with subtype 1 is 2, changed to 3


## 6. Change the rotation of the Stem
### Rotation Theta (rotaBeta) : the angle between main stem and sub branch
### Rotation Beta (rotaBeta) : the angle between two neigbour sub branch
the value of Rotation multiply by Pie is the revlution of the neighbouring sub branch.

In [117]:
from plotly.subplots import make_subplots
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}],
           [{'type': 'scatter3d'}, {'type': 'scatter3d'}]])

name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'lmax', 'value','20')
change_parameter('testXML', 'testXML', 'leaf', '2', 'RotBeta', 'value','0')
smallplant1 = CPlantBox(name, time, name) # make a plant object in python
fig1 = visual_plant_sub(smallplant1,name="0 degree rotation")

name = 'testXML'
change_parameter('testXML', 'testXML', 'leaf', '2', 'RotBeta', 'value','0.5')
smallplant2 = CPlantBox(name, time, name) # make a plant object in python
fig2 = visual_plant_sub(smallplant2,name="90 degree rotation")

name = 'testXML'
change_parameter('testXML', 'testXML', 'leaf', '2', 'RotBeta', 'value','1')
smallplant3 = CPlantBox(name, time, name) # make a plant object in python
fig3 = visual_plant_sub(smallplant3,name="180 degree rotation")

name = 'testXML'
change_parameter('testXML', 'testXML', 'leaf', '2', 'RotBeta', 'value','1.5')
smallplant4 = CPlantBox(name, time, name) # make a plant object in python
fig4 = visual_plant_sub(smallplant4,name="270 degree rotation")

fig.add_trace(fig1, 1, 1 )
fig.add_trace(fig2, 1, 2)
fig.add_trace(fig3, 2, 1)
fig.add_trace(fig4, 2, 2)

fig.update_layout(
    title_text='different RotBeta (function type)',
    height=800,
    width=800
)

fig.show()

original lmax of stem organ with subtype 1 is 20, changed to 20
original RotBeta of leaf organ with subtype 2 is 1.5, changed to 0
original RotBeta of leaf organ with subtype 2 is 0, changed to 0.5
original RotBeta of leaf organ with subtype 2 is 0.5, changed to 1
original RotBeta of leaf organ with subtype 2 is 1, changed to 1.5


## 7. 



